Tests OpenCv, VTK, ITK et Slicer 3d

In [1]:
# Imports de bibliothèques

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import dicom
import dicom_numpy
import os
import radiomics
from skimage import io
import SimpleITK as sitk
import collections
import re

In [2]:
# Création du path data
path = "C:/Users/mathi_000/jupyter/Projet/data/"

Création d'un dossier de fichiers dicom

In [3]:
DOSSIER = []
for dirpath,dirnames,filenames in os.walk(path+"001-026/dcm"):
    for file in filenames:
        DOSSIER.append(os.path.join(dirpath,file))

# for d in DOSSIER:
#     f = dicom.read_file(d)
#     I=f.pixel_array
#     plt.figure(figsize=(10,10))
#     plt.imshow(I,cmap='gray')
#     plt.show()

In [50]:
d1 = DOSSIER[50]
d2 = DOSSIER[200]
f1 = dicom.read_file(d1)
f2 = dicom.read_file(d2)
print(f1[0x54,0x16].value)
print(f2[0x54,0x16].value)
print(f1)

[(0018, 1072) Radiopharmaceutical Start Time      TM: '094845.000000'
(0018, 1074) Radionuclide Total Dose             DS: '217559997.55859'
(0018, 1075) Radionuclide Half Life              DS: '6586.2'
(0018, 1076) Radionuclide Positron Fraction      DS: '0.97'
(0054, 0300)  Radionuclide Code Sequence   1 item(s) ---- 
   (0008, 0100) Code Value                          SH: 'C-111A1'
   (0008, 0102) Coding Scheme Designator            SH: 'SNM3'
   (0008, 0104) Code Meaning                        LO: 'F^18^[^18^Fluorine]'
   (0008, 0105) Mapping Resource                    CS: 'DCMR'
   (0008, 0106) Context Group Version               DT: '20020904000000.000000'
   (0008, 010f) Context Identifier                  CS: '4020'
   ---------]
[(0018, 1072) Radiopharmaceutical Start Time      TM: '094845.000000'
(0018, 1074) Radionuclide Total Dose             DS: '217559997.55859'
(0018, 1075) Radionuclide Half Life              DS: '6586.2'
(0018, 1076) Radionuclide Positron Fraction     

In [ ]:
def SUVconversion(listDCMslices):
    

Construction d'une image 3D grâce à dicom_numpy

In [ ]:
DOSSIER = []
for dirpath,dirnames,filenames in os.walk(path+"001-026/dcm"):
    for file in filenames:
        DOSSIER.append(os.path.join(dirpath,file))

datasets = [dicom.read_file(d) for d in DOSSIER]

voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(datasets)

print(np.shape(voxel_ndarray))
print(ijk_to_xyz)

Création du masque (on prend le deuxième masque du premier patient)

In [ ]:
m = io.imread(path+"001-026/l2/kmean.tif")
print(m.shape)
print(m.T.shape)
m = m.T

# # Visualisation du masque
# for z in range(0,324):
#     plt.imshow(m[:,:,z],cmap='gray')
#     plt.show()


In [ ]:
R = radiomics.featureextractor.RadiomicsFeaturesExtractor()
print(R)
R.enableAllFeatures()
print(R.getFeatureClassNames())
R.getFeatureNames('glcm')

# Conversion array ==> simpleITK
image = sitk.GetImageFromArray(voxel_ndarray)
mask = sitk.GetImageFromArray(m)

In [ ]:
F = radiomics.glcm.RadiomicsGLCM(image, mask)
F.getContrastFeatureValue()

Test extraction masque 2.5

In [ ]:
DOSSIER_MASK = []
for dirpath,dirnames,filenames in os.walk(path+"001-026/l2/40"):
    for file in filenames:
        DOSSIER_MASK.append(os.path.join(dirpath,file))
        
#Prendre dimensions des images
DOSSIER = []
for dirpath,dirnames,filenames in os.walk(path+"001-026/dcm"):
    for file in filenames:
        DOSSIER.append(os.path.join(dirpath,file))
f = dicom.read_file(DOSSIER[0])
I=f.pixel_array
s = np.shape(I)

for i in DOSSIER_MASK:
    m = np.zeros(s)
    file = open(i)
    file.readline()
    file.readline()
    file.readline()
    for x in range(s[0]):
        for y in range(s[1]):
            r = file.read(1)
            while (r!='0' and r!='1'):
                r = file.read(1)
            r = float(r)
            if r!=0:
                m[x,y]=r
                
    flag=0            
    for y in range(168):
        for x in range(168):
            if m[x,y]==1:
                flag = 1
    if flag==1:
        plt.imshow(m, cmap='gray')
        plt.show()

In [ ]:
# Test avec fonction de francois

def getWords(text):
    return re.compile('\w+').findall(text)



list_pileFiles = []
for dirpath,dirnames,fileNames in os.walk(path+"001-026/l2/40"):
    for file in fileNames:
        list_pileFiles.append(os.path.join(dirpath,file))
list_pileFilesAsArray = []
    
first_file = list_pileFiles[0]
#get the shape of the image
with open(first_file, mode='r', encoding='utf-8') as file:
    file.readline()#first line junk data
    shapeLocalMask = getWords(file.readline()) #secondline is shape of the dcm pile
    xShape = int(shapeLocalMask[0])
    yShape = int(shapeLocalMask[1])
    mask_size = (xShape, yShape)
    
num_file = len(list_pileFiles)
mask_array = np.zeros((num_file, xShape, yShape))
fileIndex = 0
for pileFile in list_pileFiles:
    with open(pileFile, mode='r', encoding='utf-8') as file:
        file.readline() #junk line
        file.readline() #secondline is shape of the dcm pile
        file.readline() # 3 lines of junk data
        for rowIndex in range(xShape):
            for colIndex in range(yShape):
                val = file.read(1)
                while (val!='0' and val!='1'):
                    val = file.read(1)
                mask_array[fileIndex,rowIndex, colIndex] = int(val)
        fileIndex = fileIndex + 1

In [ ]:
print(mask_array.shape)
print(m.shape)

In [ ]:
for i in range (mask_array.shape[0]):
    flag=0            
    for y in range(168):
        for x in range(168):
            if mask_array[325-i,x,y]==1:
                flag = 1
    if flag==1:
        plt.imshow(mask_array[325-i, :, :], cmap='gray')
        plt.show()
    

In [ ]:
mkmean = io.imread(path+"001-026/l2/kmean.tif")
print(mkmean.shape)
for i in range (mkmean.shape[0]):
    flag=0            
    for y in range(168):
        for x in range(168):
            if mkmean[i,x,y]==1:
                flag = 1
    if flag==1:
        plt.imshow(mkmean[i, :, :], cmap='gray')
        plt.show()